In [29]:
import pandas as pd
import numpy as np
from fractions import Fraction
from collections import defaultdict



#Extract the data from excel file.
file_path = 'Data.xlsx'
data = pd.read_excel(file_path)
num_rows, num_cols = data.shape                 #Get Number of Rows and Columns
data_matrix = []                                #Initialize an Empty Matrix:

for i in range(num_rows):
    row_values = data.iloc[i].tolist()
    data_matrix.append(row_values)
data_matrix = pd.DataFrame(data_matrix)


In [30]:
#Shuffled the data, from the original one
data = pd.DataFrame(data_matrix)                                                       #Redundant Code of line, I agree
header = data.iloc[0]                                                                  # Removed the first line of the data and stored in the header variable, keep in the mind in the panda lib they extract the rows and convert it into the column because this is their method.          
data = data[1:]                                                                        #Updated the data variable by removing the first line
shuffled_data = data.sample(frac=1, random_state=42)                                    #Shuffled the data and memorise it by the random state
shuffled_data_with_header = pd.concat([header.to_frame().transpose(), shuffled_data])   #Transformed the header into frame , did transpose and put it into the shuffled data, I didn't mentioned the place so by default it took first position
shuffled_matrix = shuffled_data_with_header.values                                      #Convert back the data to NumbPy 
Training_matrix = shuffled_matrix[0:11]                                                 #Created the Train set
Test_Matrix     = shuffled_matrix[11:15]                                                #Created the Test Matrix
print(Test_Matrix)


[['rainy' 'mild' 'normal' False 'yes']
 ['sunny' 'hot' 'high' True 'no']
 ['Overcast' 'hot' 'normal' False 'yes']
 ['rainy' 'cool' 'normal' True 'no']]


In [31]:
matrix_rows, matrix_cols = Training_matrix.shape
specific_column = Training_matrix[1:, matrix_cols - 1]               #Defined the area where I have to look, I removed the first row because it take the "play", and -1 is to clear the dimension.
unique, counts = np.unique(specific_column, return_counts=True)      #Counts the yes or no along with their quantity
unique_counts = dict(zip(unique, counts))                            #Store in the dictionary   
total_samples = len(specific_column)                                 #Total samples of Yes/No
probabilities = {key: Fraction(value, total_samples) for key, value in unique_counts.items()}   #It is giving the prob. but I didn't understand this code
values_probabilities = list(probabilities.values())
Prob_NO = values_probabilities[0]
Prob_YES = values_probabilities[1] 

In [32]:
#Calculate Likhlihood Probabilities of Training Set
def Likelihood_probabilities(Training_matrix, matrix_rows, matrix_cols):
    last_col_index = matrix_cols - 1
    probabilities = []
    
    # Extracting the last column values 
    last_column_values = [row[-1] for row in Training_matrix[1:]]     # In python negative value represents the last value of the matrix or list. 
    
    unique_last_col_values = set(last_column_values)   # Getting unique values in the last column
    
    for col_index in range(matrix_cols - 1):     # Exclude the last column
        col_values = [row[col_index] for row in Training_matrix[1:]]   #  This line extracts the column values for each column. 
        unique_col_values = set(col_values)    # Makeing the set for unique values of column
        
        for value in unique_col_values:
            for last_value in unique_last_col_values:
                total_last_col_value = last_column_values.count(last_value)
                count = sum(1 for row in Training_matrix[1:] if row[col_index] == value and row[-1] == last_value)     # Gives you the count whenever the conditions is true. 
                probability = Fraction(count, total_last_col_value) if total_last_col_value != 0 else 0
                probabilities.append((value, last_value, f"{probability.numerator}/{probability.denominator}" if probability != 0 else 0))
    
    return probabilities


probabilities = Likelihood_probabilities(Training_matrix, matrix_rows, matrix_cols)
print(probabilities)





[('sunny', 'no', '2/3'), ('sunny', 'yes', '2/7'), ('rainy', 'no', '1/3'), ('rainy', 'yes', '2/7'), ('Overcast', 'no', 0), ('Overcast', 'yes', '3/7'), ('hot', 'no', '1/3'), ('hot', 'yes', '1/7'), ('cool', 'no', 0), ('cool', 'yes', '3/7'), ('mild', 'no', '2/3'), ('mild', 'yes', '3/7'), ('normal', 'no', 0), ('normal', 'yes', '4/7'), ('high', 'no', '1/1'), ('high', 'yes', '3/7'), (False, 'no', '2/3'), (False, 'yes', '4/7'), (True, 'no', '1/3'), (True, 'yes', '3/7')]


In [6]:
#Calculate the posterior probabilites of the constions of test matrix

def calculate_posterior(Test_Matrix, probabilities, Prob_NO, Prob_YES):
    results = []
    for row in Test_Matrix:
        posterior_NO = Prob_NO
        posterior_YES = Prob_YES

        for i, parameter in enumerate(row[:4]):    # It will give the the position and the parameter of the test matrix
            likelihood = next((prob[2] for prob in probabilities if prob[0] == parameter and prob[1] == 'no'), None)   #Giving the likelihood probability of "NO"

            if likelihood:
                likelihood = likelihood.split('/')  #Lookign "/" to split
                probability = Fraction(int(likelihood[0]), int(likelihood[1]))
                posterior_NO *= probability
                posterior_YES *= (1 - probability)

        prediction = "YES" if posterior_YES > posterior_NO else "NO"
        results.append(prediction)
    
    return results

# Calculate posterior probabilities for Test_Matrix
predictions = calculate_posterior(Test_Matrix, probabilities, Prob_NO, Prob_YES)
for i, prediction in enumerate(predictions):
    print(f"For condition of Row {i + 11} of the test_matrix: Play? - {prediction}")


For condition of Row 11 of the test_matrix: Play? - YES
For condition of Row 12 of the test_matrix: Play? - NO
For condition of Row 13 of the test_matrix: Play? - YES
For condition of Row 14 of the test_matrix: Play? - YES


In [76]:

def calculate_posterior_with_smoothing(Test_Matrix, probabilities, Prob_NO, num_possible_values):
    results = []
    Laplace_Prob = []
    a = 1  # Laplace smoothing parameter

    # Calculate the total count of all possible values
    v = 1
    for val in num_possible_values:
        v *= val
    
    for row in Test_Matrix:
        posterior_NO = Prob_NO

        likelihoods = []  # Collect likelihoods for all parameters in the row
        for i, parameter in enumerate(row[:4]):
            likelihood = next((prob[2] for prob in probabilities if prob[0] == parameter and prob[1] == 'no'), None)
            if likelihood:
                likelihood = likelihood.split('/')
                probability = Fraction(int(likelihood[0]), int(likelihood[1]))
                likelihoods.append(probability)

        # Calculate posterior probabilities for NO without Laplace smoothing adjustment
        for temp_prob_fraction in likelihoods:
            posterior_NO *= temp_prob_fraction

        # Apply Laplace smoothing adjustment to posterior probability for NO
        posterior_NO_smoothed = (posterior_NO.numerator + a) / (posterior_NO.denominator + (a * v))
        
        posterior_YES_smoothed = 1 - posterior_NO_smoothed

        result = "YES" if posterior_YES_smoothed > posterior_NO_smoothed else "No"
        
        results.append(result)  # Append the result for NO for the row
        Laplace_Prob.append([posterior_NO_smoothed,posterior_YES_smoothed])
    
    return results,Laplace_Prob

# Example usage:
# You need to define Test_Matrix, probabilities, Prob_NO, num_possible_values before calling the function

# Call the function
genoa,Prob = calculate_posterior_with_smoothing(Test_Matrix, probabilities, Prob_NO, num_possible_values)

# Display results for each row
for i, (result, (posterior_NO_smoothed, posterior_YES_smoothed)) in enumerate(zip(genoa, Prob)):
    print(f"For condition of Row {i + 11} of the test_matrix: Play? - {result}")
    print(f"Posterior Probability for NO: {posterior_NO_smoothed}, Posterior Probability for YES: {posterior_YES_smoothed}")



For condition of Row 11 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.037037037037037035, Posterior Probability for YES: 0.962962962962963
For condition of Row 12 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.024691358024691357, Posterior Probability for YES: 0.9753086419753086
For condition of Row 13 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.0392156862745098, Posterior Probability for YES: 0.9607843137254902
For condition of Row 14 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.030303030303030304, Posterior Probability for YES: 0.9696969696969697
